In [1]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests
import re
import urllib
import string

df = pd.read_excel("Input.xlsx")
max_row, max_col = df.shape
print(max_row)

df = df[['URL_ID','URL']]

170


In [ ]:
#text
headers = {
    "User-Agent": "Mozilla/5.0 (X11; Linux x86_64; rv:60.0) Gecko/20100101 Firefox/60.0"
}

for i in range(max_row):
    url = df.iloc[i,1]
    url_id = df.iloc[i,0]
    print(url)
    html = requests.get(url,headers=headers)
    soup = BeautifulSoup(html.content,'lxml')
    text = soup.find_all('p')
    text = [t.text for t in text]
    text = ' '.join(text)
    remove = string.punctuation
    remove = re.sub(r"[.:-]+", "", remove)
    pattern = r"[{}]".format(remove + '-') 
    text = re.sub(pattern, "", text) 
    text = re.sub("\d+", "", text)
    #text = re.sub(r'\([^)]*\)', '', text)
    #text = re.sub('[^A-Za-z0-9]+', ' ', text)
    text = text.lower()
    text = text.split()
    text = [x for x in text if len(x)>1]
    text = ' '.join(text)
    print(text)
    df.iat[i,1] = text
    df.iat[i,0] = url_id
df.to_excel('Output_with_text.xlsx', index=False)

In [ ]:
#for extracting_heading
headers = {
    "User-Agent": "Mozilla/5.0 (X11; Linux x86_64; rv:60.0) Gecko/20100101 Firefox/60.0"
}

for i in range(max_row):
    url = df.iloc[i,1]
    url_id = df.iloc[i,0]
    print(url)
    html = requests.get(url,headers=headers)
    soup = BeautifulSoup(html.content,'lxml')
    text = soup.find_all('h1')
    text = [t.text for t in text]
    text = ' '.join(text)
    text = re.sub(r'\([^)]*\)', '', text)
    text = re.sub('[^A-Za-z0-9]+', ' ', text)
    text = text.lower()
    text = text.split()
    text = [x for x in text if len(x)>1]
    text = ' '.join(text)
    print(text)
    df.iat[i,1] = text
    df.iat[i,0] = url_id
df.to_excel('Output_with_heading.xlsx', index=False)

In [ ]:
#for extracting subheading
headers = {
    "User-Agent": "Mozilla/5.0 (X11; Linux x86_64; rv:60.0) Gecko/20100101 Firefox/60.0"
}

for i in range(max_row):
    url = df.iloc[i,1]
    url_id = df.iloc[i,0]
    print(url)
    html = requests.get(url,headers=headers)
    soup = BeautifulSoup(html.content,'lxml')
    text = soup.find_all('h2')
    text = [t.text for t in text]
    text = ' '.join(text)
    text = re.sub(r'\([^)]*\)', '', text)
    text = re.sub('[^A-Za-z0-9]+', ' ', text)
    text = text.lower()
    text = text.split()
    text = [x for x in text if len(x)>1]
    text = ' '.join(text)
    print(text)
    df.iat[i,1] = text
    df.iat[i,0] = url_id

df.to_excel('Output_with_h2.xlsx', index=False)

In [2]:
article_heading = pd.read_excel('Output_with_heading.xlsx')
article_heading['article_heading'] = article_heading['URL']
article_heading = article_heading.drop('URL',axis = 1)

article_text = pd.read_excel('Output_with_text.xlsx')

article_text = article_text[['URL_ID','URL']]
article_text['text'] = article_text['URL']
article_text = article_text.drop('URL',axis=1)

article_subheading = pd.read_excel('Output_with_h2.xlsx')
article_subheading = article_subheading.fillna("is")
article_subheading['article_subheading'] = article_subheading['URL']
article_subheading = article_subheading.drop('URL',axis=1)

# create a new dataframe and add the article_text,article_heading,article_subheading
article_df = pd.merge(article_text,article_heading,on='URL_ID')
article_content = pd.merge(article_df,article_subheading,on='URL_ID')

# join heading ,text and subheading in a single column name 'content' separated by ','
article_content['content'] = article_df['article_heading'] + ',' + article_df['text'] + ',' + article_content['article_subheading']

# drop the columns 'article_heading' and 'text' and 'article_subheading'
article_content = article_content.drop(['article_heading','text','article_subheading'],axis=1)

In [3]:
article_content

,URL_ID,content
0,1,how is login logout time tracking for employee...
1,2,how does ai help to monitor retail shelf watch...
2,3,ai and its impact on the fashion industry ai i...
3,4,how do deep learning models predict old and ne...
4,5,how artificial intelligence can boost your pro...
...,...,...
165,167,"role of big data in academia,can academia rese..."
166,168,statistical methods for sales forecasting in r...
167,169,behavior based chi square model to detect data...
168,170,"what is data exfiltration,if we talk in terms ..."


In [ ]:
article_content.to_excel('article_content.xlsx', index=False)

In [4]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
from nltk.sentiment.util import *
from nltk.sentiment.vader import SentimentIntensityAnalyzer
nltk.download('vader_lexicon')
from nltk.corpus import wordnet

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\sudee\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [5]:
master_dict = pd.read_csv('./LoughranMcDonald_MasterDictionary_2020.csv')

positive_dictionary = [x for x in master_dict[master_dict['Positive'] != 0]['Word']]
negative_dictionary = [x for x in master_dict[master_dict['Negative'] != 0]['Word']]

print(f"Total positve words in dictionary are {len(positive_dictionary)}")
print(f"Total negative words in dictionary are {len(negative_dictionary)}")

Total positve words in dictionary are 354
Total negative words in dictionary are 2355


In [6]:
# making the stopword set from basic english and the given list of stopwords
#stop_words = set(stopwords.words('english'))
# upper case the stopwords
#stop_words = {word.upper() for word in stop_words}

def tokenize(text):
    text = re.sub(r'[^A-Za-z]',' ',text.upper())
    tokenized_words = word_tokenize(text)
    return tokenized_words

def remove_stopwords(content):
    content = content.upper()
    content = content.split()
    content = [word for word in content if word not in stop_words]
    content = " ".join(content)
    return content

def countfunc(store, words):
    score = 0
    for x in words:
        if(x in store):
            score = score+1
    return score

def polarity(positive_score, negative_score):
    return (positive_score - negative_score)/((positive_score + negative_score)+ 0.000001)
     

def subjectivity(positive_score, negative_score, num_words):
    return (positive_score+negative_score)/(num_words+ 0.000001)

def syllable_morethan2(word):
    if(len(word) > 2 and (word[-2:] == 'es' or word[-2:] == 'ed')):
        return False
    
    count =0
    vowels = ['a','e','i','o','u']
    for i in word:
        if(i.lower() in vowels):
            count = count +1
        
    if(count > 2):
        return True
    else:
        return False
    
def fog_index_cal(average_sentence_length, percentage_complexwords):
    return 0.4*(average_sentence_length + percentage_complexwords)

In [7]:
stopset = set(w.upper() for w in stopwords.words('english'))
# tokenize article content
article_content['tokenized_content'] = article_content['content'].apply(tokenize)
# remove stopwords
article_content['tokenized_content'] = article_content['tokenized_content'].apply(lambda x: [w for w in x if not w in stopset])

In [8]:
#count the number of positive words in the article
article_content['POSITIVE SCORE'] = article_content['tokenized_content'].apply(lambda x: countfunc(positive_dictionary, x))

#count the number of negative words in the article
article_content['NEGATIVE SCORE'] = article_content['tokenized_content'].apply(lambda x: countfunc(negative_dictionary, x))

#calculate the polarity score
article_content['POLARITY SCORE'] = article_content.apply(lambda x: polarity(x['POSITIVE SCORE'], x['NEGATIVE SCORE']), axis=1)

#calculate number of words after cleaning
article_content['num_words'] = article_content['tokenized_content'].apply(lambda x: len(x))

#calculate the subjectivity score
article_content['SUBJECTIVITY SCORE'] = article_content.apply(lambda x: subjectivity(x['POSITIVE SCORE'], x['NEGATIVE SCORE'], x['num_words']), axis=1)

In [9]:
# calculate the average sentence length
#Average Sentence Length = the number of words / the number of sentences
#first find the number of sentences
article_content['num_sentences'] = article_content['content'].apply(lambda x: len(sent_tokenize(x)))
article_content['AVG SENTENCE LENGTH'] = article_content['num_words']/article_content['num_sentences']
#fog index
#article_content['FOG INDEX'] = article_content.apply(lambda x: fog_index_cal(x['AVG SENTENCE LENGTH'], x['percentage_complexwords']), axis=1)

In [10]:
# calculate number of complex of word
article_content['num_complex_words'] = article_content['tokenized_content'].apply(lambda x: len([w for w in x if syllable_morethan2(w)]))
article_content['PERCENTAGE OF COMPLEX WORDS'] = article_content['num_complex_words']/article_content['num_words']

In [11]:
#fog index
article_content['FOG INDEX'] = article_content.apply(lambda x: fog_index_cal(x['AVG SENTENCE LENGTH'], x['PERCENTAGE OF COMPLEX WORDS']), axis=1)

In [12]:
#Average Number of Words Per Sentence = the total number of words / the total number of sentences
#first find total number of words 
article_content['total_num_words'] = article_content['tokenized_content'].apply(lambda x: sum([len(sent) for sent in x]))
#find total number of sentences
#article_content['total_num_sentences'] = article_content['num_sentences'].sum()
#calculate average number of words per sentence
article_content['AVG NUMBER OF WORDS PER SENTENCE'] = article_content['total_num_words']/article_content['num_sentences']

In [13]:
#calculate complex words count
article_content['COMPLEX WORD COUNT'] = article_content['tokenized_content'].apply(lambda x: len([w for w in x if syllable_morethan2(w)]))

# calculate WORD COUNT
article_content['WORD COUNT'] = article_content['tokenized_content'].apply(lambda x: len(x))

In [14]:
article_content['SYLLABLE PER WORD'] = article_content['tokenized_content'].apply(lambda x: sum([len(w) for w in x if syllable_morethan2(w)]))

In [15]:
import re
article_content['PERSONAL PRONOUN'] = article_content['content'].apply(lambda x: len(re.findall(r'\b(i|we|my|ours|us)\b', x)))

In [16]:
pd.set_option('display.max_columns',None)

In [18]:
# CALCULATE AVERAGE WORD LENGTH
article_content['num_char'] = article_content['content'].apply(lambda x: sum([len(w) for w in x]))

article_content['AVG WORD LENGTH'] = article_content['num_char']/article_content['total_num_words']

In [20]:
article_content.head()

,URL_ID,content,tokenized_content,POSITIVE SCORE,NEGATIVE SCORE,POLARITY SCORE,num_words,SUBJECTIVITY SCORE,num_sentences,AVG SENTENCE LENGTH,num_complex_words,PERCENTAGE OF COMPLEX WORDS,FOG INDEX,total_num_words,AVG NUMBER OF WORDS PER SENTENCE,COMPLEX WORD COUNT,WORD COUNT,SYLLABLE PER WORD,PERSONAL PRONOUN,num_char,AVG WORD LENGTH
0,1,how is login logout time tracking for employee...,"[LOGIN, LOGOUT, TIME, TRACKING, EMPLOYEES, OFF...",4,5,-0.111111,412,0.021845,24,17.166667,175,0.424757,7.036570,2693,112.208333,175,412,1498,4,4305,1.598589
1,2,how does ai help to monitor retail shelf watch...,"[AI, HELP, MONITOR, RETAIL, SHELF, WATCHES, IN...",9,6,0.200000,377,0.039788,27,13.962963,177,0.469496,5.772984,2597,96.185185,177,377,1539,2,3978,1.531767
2,3,ai and its impact on the fashion industry ai i...,"[AI, IMPACT, FASHION, INDUSTRY, AI, FAST, FASH...",31,23,0.148148,1060,0.050943,75,14.133333,558,0.526415,5.863899,7462,99.493333,558,1060,4869,13,11368,1.523452
3,4,how do deep learning models predict old and ne...,"[DEEP, LEARNING, MODELS, PREDICT, OLD, NEW, DR...",6,1,0.714286,255,0.027451,14,18.214286,134,0.525490,7.495910,1791,127.928571,134,255,1183,1,2765,1.543830
4,5,how artificial intelligence can boost your pro...,"[ARTIFICIAL, INTELLIGENCE, BOOST, PRODUCTIVITY...",14,13,0.037037,399,0.067669,32,12.468750,193,0.483709,5.180984,2698,84.312500,193,399,1717,20,4285,1.588213


In [21]:
article_content.to_excel('added_variables.xlsx', index=False)

In [23]:
article_content.head(2)

,URL_ID,content,tokenized_content,POSITIVE SCORE,NEGATIVE SCORE,POLARITY SCORE,num_words,SUBJECTIVITY SCORE,num_sentences,AVG SENTENCE LENGTH,num_complex_words,PERCENTAGE OF COMPLEX WORDS,FOG INDEX,total_num_words,AVG NUMBER OF WORDS PER SENTENCE,COMPLEX WORD COUNT,WORD COUNT,SYLLABLE PER WORD,PERSONAL PRONOUN,num_char,AVG WORD LENGTH
0,1,how is login logout time tracking for employee...,"[LOGIN, LOGOUT, TIME, TRACKING, EMPLOYEES, OFF...",4,5,-0.111111,412,0.021845,24,17.166667,175,0.424757,7.036570,2693,112.208333,175,412,1498,4,4305,1.598589
1,2,how does ai help to monitor retail shelf watch...,"[AI, HELP, MONITOR, RETAIL, SHELF, WATCHES, IN...",9,6,0.200000,377,0.039788,27,13.962963,177,0.469496,5.772984,2597,96.185185,177,377,1539,2,3978,1.531767


In [24]:
variable_computing = article_content.drop(['content','tokenized_content','num_words','num_sentences','num_complex_words','total_num_words','num_char'], axis=1)

In [25]:
variable_computing.head(3)

,URL_ID,POSITIVE SCORE,NEGATIVE SCORE,POLARITY SCORE,SUBJECTIVITY SCORE,AVG SENTENCE LENGTH,PERCENTAGE OF COMPLEX WORDS,FOG INDEX,AVG NUMBER OF WORDS PER SENTENCE,COMPLEX WORD COUNT,WORD COUNT,SYLLABLE PER WORD,PERSONAL PRONOUN,AVG WORD LENGTH
0,1,4,5,-0.111111,0.021845,17.166667,0.424757,7.036570,112.208333,175,412,1498,4,1.598589
1,2,9,6,0.200000,0.039788,13.962963,0.469496,5.772984,96.185185,177,377,1539,2,1.531767
2,3,31,23,0.148148,0.050943,14.133333,0.526415,5.863899,99.493333,558,1060,4869,13,1.523452


In [26]:
output_file = pd.read_excel('Output_Data_Structure.xlsx')

In [30]:
variable_computing['URL'] = output_file['URL']

In [32]:
variable_computing = variable_computing[['URL_ID','URL','POSITIVE SCORE','NEGATIVE SCORE','POLARITY SCORE','SUBJECTIVITY SCORE','AVG SENTENCE LENGTH','PERCENTAGE OF COMPLEX WORDS','FOG INDEX','AVG NUMBER OF WORDS PER SENTENCE','COMPLEX WORD COUNT','WORD COUNT','SYLLABLE PER WORD','PERSONAL PRONOUN','AVG WORD LENGTH']]

In [35]:
variable_computing.to_excel('output_data.xlsx', index=False)